In [1]:
use Modern::Perl;
use PDL;
use Time::HiRes qw(time);

### Some functions to compare

In [2]:
sub mcpi_0 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        $count += 1 if (sqrt($a*$a + $b*$b) < 1);
    }

    return 4*$count/$runs;
}

sub mcpi_1 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        $count += 1 if (($a*$a + $b*$b) < 1);
    }

    return 4*$count/$runs;
}

sub mcpi_2 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        
        $count += 1 if (($a*$a + $b*$b) > 1);
    }

    return 4*(1-$count/$runs);
}

sub mcpi_3 {
    my $runs  = shift;
    my ($a,$b);
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        ($a,$b) = (rand,rand);
        
        $count += 1 if (($a*$a + $b*$b) > 1);
    }

    return 4*(1-$count/$runs);
}

sub mcpi_pdl {
# two arguments
    my $runs      = shift;
    my $veclength = shift;
    my $count      = 0;
    my ($a,$b);

    foreach (1 .. $runs){
        $a = random($veclength);
        $b = random($veclength);
        $count = $count + sum(($a*$a+$b*$b) > 1);
    }


    my $total = $runs*$veclength;
    return 4*(1-$count/$total);
}


In [3]:
printf("%10.6f\n", mcpi_0(100000));
printf("%10.6f\n", mcpi_1(100000));
printf("%10.6f\n", mcpi_2(100000));
printf("%10.6f\n", mcpi_3(100000));
printf("%10.6f\n", mcpi_pdl(100,1000));

  3.144680
  3.146160
  3.141920
  3.142440
  3.133720


1


In [4]:
use Benchmark qw(cmpthese);

cmpthese(1000, {
    'mcpi_0' => sub { mcpi_0(1E4) },
    'mcpi_1' => sub { mcpi_1(1E4) },
    'mcpi_2' => sub { mcpi_2(1E4) },
    'mcpi_3' => sub { mcpi_3(1E4) },
    'mcpi_pdl' => sub { mcpi_pdl(1,1E4) },
});
0;

            (warning: too few iterations for a reliable count)
           Rate   mcpi_0   mcpi_1   mcpi_2   mcpi_3 mcpi_pdl
mcpi_0    290/s       --     -11%     -14%     -24%     -93%
mcpi_1    325/s      12%       --      -4%     -15%     -93%
mcpi_2    337/s      16%       4%       --     -12%     -92%
mcpi_3    383/s      32%      18%      14%       --     -91%
mcpi_pdl 4348/s    1400%    1239%    1191%    1035%       --


0


In [5]:
# start the clock
my $ta = time;
my $pi = mcpi_pdl(1E3,1E5);
say "time: ", time - $ta;
printf ("%10.7f\n", $pi);


time: 2.2760808467865
 3.1418271


1


## Some graphs to play with

In [22]:
my @runs = (1 .. 500);
# first run
my @pi_0 = ();
push @pi_0, mcpi_pdl(1,1E5) foreach @runs;

#second run
my @pi_1 = ();
push @pi_1, mcpi_pdl(1E1,1E5) foreach @runs;

my $PI = 3.14159265359;
IPerl->load_plugin( "ChartClicker" );
my $cc = Chart::Clicker->new(width => 523, height => 384);

my $series_0 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@pi_0],
);
my $series_1 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@pi_1],
);

my $series_pi = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [map{$PI} @runs],
);

# build the dataset
my $dataset = Chart::Clicker::Data::DataSet->new(
    series  =>      [ $series_0, $series_1, $series_pi ],
);

$cc->add_to_datasets($dataset);

$cc;


## Visualize MSE

In [23]:
my @avg_0=();
my @avg_1=();

my ($ra_0,$ra_1) = (0,0); 
foreach my $i (@runs){
    $ra_0 += $pi_0[$i-1];
    push @avg_0, $ra_0/$i;

    $ra_1 += $pi_1[$i-1];
    push @avg_1, $ra_1/$i;
}


my $cc = Chart::Clicker->new(width => 523, height => 384);

my $series_0 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@avg_0],
);
my $series_1 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@avg_1],
);

my $series_pi = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [map{$PI} @runs],
);

# build the dataset
my $dataset = Chart::Clicker::Data::DataSet->new(
    series  =>      [ $series_0, $series_1, $series_pi ],
);

$cc->add_to_datasets($dataset);

$cc;
